# TOPOS INTERN ASSIGNMENT

We are scraping the first table from "wikipedia", the link of datasource is from "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", it contains several of table, we determine to extract the first one for this assignment. 

In this assignment, the python3 will be used to extract, clean and generate csv file for the next part in "BigQuery table". 

In [1]:
#load libraries
import numpy as np
import requests
import bs4 as bs
import pandas as pd
import os
import codecs

In [2]:
resp = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')
soup = bs.BeautifulSoup(resp.text, 'lxml')
tables = soup.findAll('table', { 'class' : 'wikitable sortable' })[0]

rows=tables.findAll("tr")[1:315]
row_lengths=[len(r.findAll(['th','td'])) for r in rows]
ncols=max(row_lengths)
nrows=len(rows)
data=[]
for i in range(nrows):
    rowD=[]
    for j in range(ncols):
        rowD.append('')
    data.append(rowD)
    
# process html
for i in range(len(rows)):
    row=rows[i]
    rowD=[]
    cells = row.findAll(["td","th"])
    for j in range(len(cells)):
        cell=cells[j]
            
        #lots of cells span cols and rows so lets deal with that
        cspan=int(cell.get('colspan',1))
        rspan=int(cell.get('rowspan',1))
        for k in range(rspan):
            for l in range(cspan):
                data[i+k][j+l]+=cell.text
    
    data.append(rowD)

In [3]:
# cleaning

frame1 = pd.DataFrame(data)
frame1 = frame1.iloc[:314, 1:11]

# remove "\n" symbol
frame1 = frame1.replace('\n','',regex=True)
frame1.columns = ['city','state','2018_estimate','2010_census','change','2016_land_area1','2016_land_area2','2016_pop_den1', '2016_pop_den2', 'loc']
frame1['city'] = frame1['city'].str.split('[').str[0]

In [4]:
display(frame1.head())

,city,state,2018_estimate,2010_census,change,2016_land_area1,2016_land_area2,2016_pop_den1,2016_pop_den2,loc
0,New York City,New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,Houston,Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


In [5]:
# Create CSV file
frame1.to_dense().to_csv("assignment.csv", index = False, sep=',', encoding='utf-8')